In [1]:
from pypbc import *
import hashlib
import time

In [2]:
stored_params = """type a
q 8780710799663312522437781984754049815806883199414208211028653399266475630880222957078625179422662221423155858769582317459277713367317481324925129998224791
h 12016012264891146079388821366740534204802954401251311822919615131047207289359704531102844802183906537786776
r 730750818665451621361119245571504901405976559617
exp2 159
exp1 107
sign1 1
sign0 1
"""

params = Parameters(param_string=stored_params)
pairing = Pairing(params)

In [13]:
def setup():
    g = Element.random(pairing, G1)
    h = Element.random(pairing, G1)
    zeta = Element.random(pairing, G2)
    
    return g, h, zeta

g, h, zeta = setup()
zeta

039FB631E6819564462866E8EFF0D5B0E596039DB8B2186A240CBDFDCEF7541CE0A6798303852A3EC6DC9BF24E8747137CB7146369BE08900280D5BDD25E927162

In [14]:
def init(num, tg):
    t_m, m = [], []
    for i in range(num):
        m.append(Element(pairing, GT))
        t_m.append(Element.random(pairing, G2))
        
    m = [pairing.apply(tg, t_m[i]) for i in range(num)]
    
    return m

m_a = init(10, g)
m_a

[(0x0CC9B308EED5EEC3163CD277D9EE4ABE239E3074CF31665E6FA3E3CE85DB3FB09F48FE0CBE6CB4426DE17590F546B698F5420CC5A08921A3BAF7638ECF37D3D8, 0x04FA8AE2FFEC39DF1ECB3577FA59E834F9E606E5A481C83228F56D819F454E239D7C857F6583EEA483DC2DBB108080DEB0834047D2571F3EB8FF7039755B8F42),
 (0x3EB0AD13838F59858326EDCDC94AA86495643EB6FB8067776D07BCFF4D8A46F4F1E8F2B192F697706386040CA6CDD982CCC810BCD6278EE55D6B7560AD443E02, 0x7DF7D39D3DBB7461C97D78D623322B7ACF233540D7E9EC9B57DA58A45F5111E143230C4887271238B764CAA0EB99BB99543EC89499A727CAB2FD0D7340304620),
 (0x6706932828F0BDE506A17FA4E9EBF0A7B2818B536A0E3C75BD6574A9AC137747B85E1DFFF69C68006B4EB287E4CC46ABF0286976FE8C3B502E908BC6118A409C, 0x9B660E2558CB1AF3D46F0F435B718707C8EAC6CBABCBC37BA1B4C2CD86B5A53CF6AC16512E22A3E7C1AAC2F9A23A0EE15D3E633D8A816DE878D2CC76665CFE27),
 (0x6C9B464EC249D9BD4EA8E44F56A45AA9E2F156AFF917D4F9CB5BB2FBDD93C1568FD393EAC9B724357A5C72A013AF0513E25BBBE7C2632E9BE47DEBE77B7688B1, 0x467E0DB762E63F2B43B84AFF4D85D8A3D333C4688ABFE017D4E114BEE8BAB8E

In [16]:
z_time = 0.0
p_time = 0.0
c = Element(pairing, GT)

for i in range(100):
    a = Element.random(pairing, G1)
    b = Element.random(pairing, G1)
    r = Element.random(pairing, Zr)
    
    st = time.time()
    d = m_a[0]**r
    z_time += time.time() - st
    
    start = time.time()
    c = pairing.apply(a, b)
    p_time += time.time() - start

print('Z Time: ', z_time / 100)
print('P Time: ', p_time / 100)

Z Time:  0.0006990814208984375
P Time:  0.0021479058265686034


In [139]:
def dkg(t_zeta):
    dpk = Element(pairing, G2)
    
    alpha = Element.random(pairing, Zr)
    dpk = t_zeta**alpha
    
    return dpk, alpha

dpk, alpha = dkg(zeta)
alpha

0x1D05E9EEF0D280DF6EC7C2D926AE1AA093417881

In [140]:
def ukg(tg, th):
    sk, pk = [], []
    for i in range(6):
        sk.append(Element.random(pairing, Zr))

    pk = [(tg**sk[i]) * (th**sk[i+1]) for i in range(0, 6, 2)]
    
    return sk, pk

sk, pk = ukg(g, h)
sk

[0x7D5ED1E98CBA4FD50CE052042B9BB31F40D8E7F5,
 0x30DA0B065E32FD6FF385BCC96CB02DDA05989DD2,
 0x2892CBBCDF1BB51DC50CA94319A8CE9FB19DC092,
 0x781E5744ED3595D856DB8BF62CA250776B9DFD97,
 0x5180A6FE70C2E3F2CCA82D398B679C50E8C415DD,
 0x31F7E346B2A04F4550394F2F6018959F47C82D52]

In [141]:
def tkg(tdpk, tsk):
    tk = []
    for i in range(6):
        tk.append(Element(pairing, G2))
    
    tk = [tdpk**tsk[i] for i in range(6)]
    
    return tk

tk = tkg(dpk, sk1)
tk

[036230A4C50D65081069A5D80154967760C74172B9DBA7596E044BDB5166F245B6CC6E7AE3CEC1526EA09D664223DB1C162E9C41E00D5FD1E4228389406217E3FA,
 0360E6D1DC26A28C819057DC93EA2D48AA745F9CA611B1BB491E682207B11D5507FB06EDA5CFD911A2651BB5E9129FB07BE3400D37BF7423C8F5C370F7789221F1,
 0346BC45BC94C839E7FCEEC21560CDB98A2ED3DB9B07FBFE0F244DFB9FC1D455DAF5ED623F65E99700AF1EFCDF0853BFE5882E02FE84B79062E86FD49CE05DD47B,
 0329FFC8DD403CD8CDDE0156D92CB30B9C30C5EFF15AF62916873C9FFFC6F55AA96BCB13F3F9D4F2EABA168EE9E706CB47EFCB95F938A0C8AF29EF205832831E02,
 03246D4B455B419181653B3CFDFF962AD866DEFAA43A06290A1CDD4793D3E2CA3475EBF3F5DFC0343CFB79FDCD505E60DE6BF968F9E0AA713E0FBFD66FEF7620F6,
 0392E0C14D5D3211706C2885F49BC46936735ED1B58DCAB6F0D6C73984E054A64D569053C21FB26C12CA7855425573C34B4AF9FA1F963495EA15D549FBCA863E94]

In [142]:
sk1, pk1 = ukg(g, h)
tk1 = tkg(dpk, sk1)

sk2, pk2 = ukg(g, h)
tk2 = tkg(dpk, sk2)
tk1

[020100E71F192786E8666D6D7571729144109F75B845DAABE373D07ED0B1AA7F960226936E1C7E02BD26C0AE718AD6F6628E790668190C994A3F50CD3832BCDBE2,
 02348FFC1347C89539B54A85D9542191DC3CC1B11E7BBB998D331D5AF6E121A791E7E2F2B44A0FADDF181F551C8DCB5AB276F8AD1A97DAAA6A269C607EA183B5D6,
 02467F871F7F34C58ECE6D676282666F42BB9677C1A531B00F8825B4945F632B0A12B98E5833EF71AB713876030555782A4984ECCC6F76900378FEA29799515A6D,
 03035F4BA0D7405637823259CBD43A9B7656FA0CA8A7910C97800B531E7F7A3A817A388F6D340428F6CB25120E9B6FFAC33E92808DD6DDC2A1C8A096FBFDDE5604,
 029E20842CD02A0C7BD601C0243DE55161E3CA9F134E75632E65F3AF2E469ABF10759C7B5356C9DB827A36AC2C135E8B355282256ACF04823B88491B915EB15300,
 0301A60B4DDF62DDA2D1A831DDA19C89BFC624B218B607B26303193EB0F2D69587873FF143856CF5716E7CE8999663F269AFEA43A477F15EE3C1CA91907DFBDD12]

In [143]:
def get_hash(c1, c2, c3):
    h_in = str(c1) + str(c2) + str(c3)
    sha512 = hashlib.sha512()
    sha512.update(h_in.encode('utf8'))
    hv = sha512.hexdigest()
    ev = Element.from_hash(pairing, Zr, hv)
    
    return ev

r = Element.random(pairing, Zr)
        
w1 = g**r
w2 = h**r

x = Element(pairing, GT)
x = pairing.apply(pk1[0], zeta)

theta = get_hash(w1, w2, x)
theta

0x6134333438353162663366663536343861613039

In [205]:
def enc(num, tg, th, t_zeta, tpk, ttk, tsk, tm):
    one = Element.one(pairing, Zr)
    start = time.time()
    c = []
    for i in range(num):
        r = Element.random(pairing, Zr)
        
        w1 = tg**r
        w2 = th**r

        e = pairing.apply(tpk[0]**r, zeta)
        x = (pairing.apply(tpk[0]**r, zeta)) * tm[i] # e((g^s * h^t)^r, zeta)
        
        print((e**(-one)*x) == tm[i])
        
        e1 = pairing.apply(w1, ttk[0])
        e2 = pairing.apply(w2, ttk[1])
        
        theta = get_hash(w1, w2, x)
        y = pairing.apply((tpk[1]**r) * (tpk[2]**theta)**r, zeta)
        
        e3 = pairing.apply(w1, ttk[2])
        e4 = pairing.apply(w2, ttk[3])
        e5 = pairing.apply(w1, ttk[4])
        e6 = pairing.apply(w2, ttk[5])
        e7 = e3 * e4 * (e5**theta) * (e6**theta)
#         print((y**alpha) == e7)
        
        c.append([w1, w2, x, y])
    
    print("Enc Time: ", time.time() - start)
    return c

c1 = enc(10, g, h, zeta, pk1, tk1, sk1, m_a)
c2 = enc(10, g, h, zeta, pk2, tk2, sk2, m_a)


True
True
True
True
True
True
True
True
True
True
Enc Time:  0.6691927909851074
True
True
True
True
True
True
True
True
True
True
Enc Time:  0.8716683387756348


In [204]:
one = Element.one(pairing, Zr)
print((c1[0][2] * (c2[0][2]**-one))**alpha)
print((c1[0][3] * (c2[0][3]**-one)))

(0x27A7A280F3C22946AD32EA7314977E1F5B33990CD13CC4FDA9C840A93ACC074556411FD9A7574D0E9F81DDF27B1D169EF07B52E4CCA294BF0D29F4C9F409637E, 0x22CA5752B43340B33505DE142BE4E31816C1A846E109EEB012D12444A0DAA208F33ED8ED26974C2900767C88E7668C16B3C047FA1E4B11829E368A17C9908FEC)
(0x81BBD7D6A29321FAB6BFC947CCAA1A828C17FEAD24DD7BDAAE22D868E64B6B9D509B5CB27AA4B7E627880080B596F68550322A4876FF4542BD0E81E20B24B269, 0x4789ABC8CC3598821B6D67D56BB2D4FA232D576E1B8DC7EBCC756E9DC8485D578E11EEFEDAFE0EBDD078F7E8B367EE471002C5EB7AE301E800AD9563977AEC34)


In [201]:
def dec(tsk, tc):
    m = []
    one = Element.one(pairing, Zr)
    
    for i in range(len(tc)):
        w1 = tc[i][0]
        w2 = tc[i][1]
        x = tc[i][2]
        y = tc[i][3]
        
        theta = get_hash(w1, w2, x)
        ty1 = (w1**(tsk[2] + theta*tsk[4]))
        ty2 = (w2**(tsk[3] + theta*tsk[5]))
        e1 = pairing.apply((ty1 * ty2), zeta)
        
        if y != e1:
            return None
        else:
            tx = (w1**tsk[0]) * (w2**tsk[1])
            e2 = pairing.apply(tx, zeta)
            m.append(x * (e2**(-one)))
#             print(m)
    
    return m

m_b = dec(sk1, c1)
for i in range(len(m_a)):
    print(m_a[i] == m_b[i])

True
True
True
True
True
True
True
True
True
True


In [211]:
def IVgen(tc1, tc2, ttk1, ttk2):
    v = []
    count = 0
    one = Element.one(pairing, Zr)
    
    for i in range(len(c1)):
        for j in range(len(c2)):
            print(i, j)
            
            omega1 = Element.random(pairing, Zr)
            omega2 = Element.random(pairing, Zr)
        
            w1_1 = tc1[i][0]
            w2_1 = tc1[i][1]
            x_1 = tc1[i][2]
            y_1 = tc1[i][3]
        
            w1_2 = tc2[j][0]
            w2_2 = tc2[j][1]
            x_2 = tc2[j][2]
            y_2 = tc2[j][3]
    
            v1 = (x_1 * (x_2**(-one)))**omega1
        
            e1 = pairing.apply(w1_1, ttk1[0])
            e2 = pairing.apply(w2_1, ttk1[1])
            e3 = pairing.apply(w1_2, ttk2[0])
            e4 = pairing.apply(w2_2, ttk2[1])
        
            v2 = ((e1 * e2) * ((e3 * e4)**(-one)))**omega1
        
            print("x: ", (v1**alpha) == v2)
            if (v1**alpha) == v2:
                count += 1
        
            v3 = (y_1 * (y_2**(-one)))**omega2
        
            theta1 = get_hash(w1_1, w2_1, x_1)
            theta2 = get_hash(w1_2, w2_2, x_2)
        
            e5 = pairing.apply(w1_1, ttk1[2])
            e6 = pairing.apply(w2_1, ttk1[3])
            e7 = (pairing.apply(w1_1, ttk1[4]))
            e8 = (pairing.apply(w2_1, ttk1[5]))
            temp_y1 = e5 * e6 * (e7**theta1) * (e8**theta1)
        
            e9 = pairing.apply(w1_2, ttk2[2])
            e10 = pairing.apply(w2_2, ttk2[3])
            e11 = (pairing.apply(w1_2, ttk2[4]))
            e12 = (pairing.apply(w2_2, ttk2[5]))
            temp_y2 = e9 * e10 * (e11**theta2) * (e12**theta2)
        
            print("y: ", (y_2**alpha) == temp_y2)
        
            v4 = (temp_y1 * (temp_y2**(-one)))**omega2
            v.append([v1, v2, v3, v4])
    print("count: ", count)
    return v

v = IVgen(c1, c2, tk1, tk2)
print(len(v))

0 0
x:  True
y:  True
0 1
x:  False
y:  True
0 2
x:  False
y:  True
0 3
x:  False
y:  True
0 4
x:  False
y:  True
0 5
x:  False
y:  True
0 6
x:  False
y:  True
0 7
x:  False
y:  True
0 8
x:  False
y:  True
0 9
x:  False
y:  True
1 0
x:  False
y:  True
1 1
x:  True
y:  True
1 2
x:  False
y:  True
1 3
x:  False
y:  True
1 4
x:  False
y:  True
1 5
x:  False
y:  True
1 6
x:  False
y:  True
1 7
x:  False
y:  True
1 8
x:  False
y:  True
1 9
x:  False
y:  True
2 0
x:  False
y:  True
2 1
x:  False
y:  True
2 2
x:  True
y:  True
2 3
x:  False
y:  True
2 4
x:  False
y:  True
2 5
x:  False
y:  True
2 6
x:  False
y:  True
2 7
x:  False
y:  True
2 8
x:  False
y:  True
2 9
x:  False
y:  True
3 0
x:  False
y:  True
3 1
x:  False
y:  True
3 2
x:  False
y:  True
3 3
x:  True
y:  True
3 4
x:  False
y:  True
3 5
x:  False
y:  True
3 6
x:  False
y:  True
3 7
x:  False
y:  True
3 8
x:  False
y:  True
3 9
x:  False
y:  True
4 0
x:  False
y:  True
4 1
x:  False
y:  True
4 2
x:  False
y:  True
4 3
x:  False
y

In [134]:
def pTest(num, tc1, tc2, tpk1, tpk2, ttk1, ttk2, tdpk):
    v = []
    start = time.time()

    for i in range(num):
        e1 = pairing.apply(tpk1[0], tdpk)
        e2 = pairing.apply(g, ttk1[0])
        e3 = pairing.apply(h, ttk1[1])
        b1 = (e1 == (e2 * e3))
        
        e4 = pairing.apply(tpk1[1], tdpk)
        e5 = pairing.apply(g, ttk1[2])
        e6 = pairing.apply(h, ttk1[3])
        b2 = (e4 == (e5 * e6))
        
        e7 = pairing.apply(tpk1[2], tdpk)
        e8 = pairing.apply(g, ttk1[4])
        e9 = pairing.apply(h, ttk1[5])
        b3 = (e7 == (e8 * e9))
        
        e10 = pairing.apply(tpk2[0], tdpk)
        e11 = pairing.apply(g, ttk2[0])
        e12 = pairing.apply(h, ttk2[1])
        b4 = (e10 == (e11 * e12))
        
        e13 = pairing.apply(tpk2[1], tdpk)
        e14 = pairing.apply(g, ttk2[2])
        e15 = pairing.apply(h, ttk2[3])
        b5 = (e13 == (e14 * e15))
        
        e16 = pairing.apply(tpk2[2], tdpk)
        e17 = pairing.apply(g, ttk2[4])
        e18 = pairing.apply(h, ttk2[5])
        b6 = (e16 == (e17 * e18))
        
        b7 = (b1 and b2 and b3 and b4 and b5 and b6)
        if b7:
            IVgen(tc1, tc2, ttk1, ttk2)
    
pTest(10, c1, c2, pk1, pk2, tk1, tk2, dpk)

True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True
True True True True True True


In [212]:
def dTest(tv, t_alpha):
    if((tv[2]**t_alpha) != tv[3]):
        return None
    else:
        if((tv[0]**t_alpha) != tv[1]):
            return 0
        else:
            return 1

# print(v)
for i in range(len(v)):
    print(dTest(v[i], alpha))

1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1


In [13]:
json_file = './discriminator.json'
with open(json_file, 'r') as f:
    d_info = json.load(f)

dsk_s = d_info['dsk']
dsk_v = int(dsk_s, 16)
dsk = Element(pairing, Zr, value=dsk_v)
dsk

0x568BFFE8FD340F772F08C6D2DBA013E9E14787B4